In [1]:
from pandas import *
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

import math
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from numpy import *
from sklearn.model_selection import cross_val_score


In [2]:
train=read_csv("Consumer_Complaints_train.csv")
xtest=read_csv("Consumer_Complaints_test_share.csv")
ytest=read_csv('sample_submission.csv')

train


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478416,2015-06-30,Bank account or service,Checking account,Problems caused by my funds being low,NaN,NaN,NaN,JPMorgan Chase & Co.,TX,77008,NaN,NaN,Phone,2015-07-01,Closed with explanation,Yes,No,1445488
478417,2013-06-20,Credit card,NaN,Collection practices,NaN,NaN,NaN,Citibank,AL,36111,NaN,NaN,Phone,2013-06-21,Closed with explanation,Yes,No,438011
478418,2014-09-09,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,TX,79201,NaN,NaN,Web,2014-09-09,Closed with explanation,Yes,No,1020651
478419,2012-09-21,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"Select Portfolio Servicing, Inc",CA,91765,NaN,NaN,Web,2012-09-24,Closed with explanation,Yes,No,157524


In [3]:
for col in ['Date received','Date sent to company']:
    train[col]=to_datetime(train[col],infer_datetime_format=True)
    xtest[col]=to_datetime(xtest[col],infer_datetime_format=True)

In [4]:
train['day_diff']=to_numeric(train['Date sent to company']-train['Date received'])
xtest['day_diff']=to_numeric(xtest['Date sent to company']-xtest['Date received'])

In [5]:
train = train.drop(['Date sent to company','Date received','Complaint ID','ZIP code','State'], axis=1)
xtest = xtest.drop(['Date sent to company','Date received','Complaint ID','ZIP code','State'], axis=1)

In [6]:
for col in train.columns:
    #print(col,train[col].value_counts())
    if train[col].isnull().sum() > 0:
        print(col,train[col].isnull().sum())
    
    #print(col,train[col].nunique())

Sub-product 138473
Sub-issue 292625
Consumer complaint narrative 403327
Company public response 388029
Tags 411215
Consumer consent provided? 342934


In [7]:
train['Product'].value_counts()

Mortgage                   156175
Debt collection             86544
Credit reporting            81115
Credit card                 57358
Bank account or service     54403
Consumer Loan               18599
Student loan                14918
Money transfers              3349
Payday loan                  3219
Prepaid card                 2226
Other financial service       507
Virtual currency                8
Name: Product, dtype: int64

In [8]:
#product-10,, 
#Submitted via 6
#Company response to consumer 7

In [9]:
train.columns

Index(['Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'Tags', 'Consumer consent provided?', 'Submitted via',
       'Company response to consumer', 'Timely response?',
       'Consumer disputed?', 'day_diff'],
      dtype='object')

In [10]:
#coz v.few unique val and no nas 
for col in ['Product','Company response to consumer','Submitted via','Timely response?']:
    store_dum=get_dummies(train[col],drop_first=True)
    train=concat([store_dum,train],axis=1)
    train.drop([col],axis=1,inplace=True)

In [11]:
train

,Yes,Fax,Phone,Postal mail,Referral,Web,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,...,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,Tags,Consumer consent provided?,Consumer disputed?,day_diff
0,1,0,0,0,0,1,1,0,0,0,...,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,Older American,NaN,No,86400000000000
1,1,0,0,0,1,0,0,0,0,0,...,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,NaN,NaN,No,518400000000000
2,1,0,0,0,1,0,0,0,1,0,...,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,NaN,NaN,No,1814400000000000
3,1,0,0,0,0,1,0,1,0,0,...,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,Older American,Consent provided,No,0
4,1,0,0,0,0,1,1,0,0,0,...,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,NaN,NaN,No,691200000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478416,1,0,1,0,0,0,1,0,0,0,...,Checking account,Problems caused by my funds being low,NaN,NaN,NaN,JPMorgan Chase & Co.,NaN,NaN,No,86400000000000
478417,1,0,1,0,0,0,1,0,0,0,...,NaN,Collection practices,NaN,NaN,NaN,Citibank,NaN,NaN,No,86400000000000
478418,1,0,0,0,0,1,1,0,0,0,...,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,NaN,NaN,No,0
478419,1,0,0,0,0,1,1,0,0,0,...,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"Select Portfolio Servicing, Inc",NaN,NaN,No,259200000000000


In [12]:
xtest.isnull().sum()

Product                              0
Sub-product                      34752
Issue                                0
Sub-issue                        73060
Consumer complaint narrative    101049
Company public response          96830
Company                              0
Tags                            102735
Consumer consent provided?       85742
Submitted via                        1
Company response to consumer         0
Timely response?                     0
day_diff                             0
dtype: int64

In [13]:
xtest.nunique()

Product                            12
Sub-product                        47
Issue                              94
Sub-issue                          67
Consumer complaint narrative    18438
Company public response            10
Company                          2237
Tags                                3
Consumer consent provided?          4
Submitted via                       6
Company response to consumer        7
Timely response?                    2
day_diff                          290
dtype: int64

In [14]:
xtest['Submitted via']=xtest['Submitted via'].fillna(xtest['Submitted via'].mode()[0])


In [15]:
#coz v.few unique val and no nas 
for col in ['Product','Company response to consumer','Submitted via','Timely response?']:
    store_dum1=get_dummies(xtest[col],drop_first=True)
    xtest=concat([store_dum1,xtest],axis=1)
    xtest.drop([col],axis=1,inplace=True)

In [16]:
xtest

,Yes,Fax,Phone,Postal mail,Referral,Web,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,...,Virtual currency,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,Tags,Consumer consent provided?,day_diff
0,1,0,0,0,0,1,1,0,0,0,...,0,Cashing a check without an account,Deposits and withdrawals,NaN,NaN,NaN,Bank of America,NaN,NaN,-86400000000000
1,1,0,0,0,0,1,1,0,0,0,...,0,Credit card,Cont'd attempts collect debt not owed,Debt was paid,NaN,NaN,"National Credit Adjusters, LLC",NaN,Consent not provided,0
2,1,0,0,0,0,1,0,0,0,0,...,0,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,NaN,NaN,86400000000000
3,1,0,0,1,0,0,0,0,1,0,...,0,NaN,Unable to get credit report/credit score,Problem getting report or credit score,NaN,Company chooses not to provide a public response,"TransUnion Intermediate Holdings, Inc.",Older American,NaN,432000000000000
4,1,0,0,0,0,1,1,0,0,0,...,0,FHA mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119601,1,0,0,0,0,1,0,1,0,0,...,0,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,NaN,NaN,518400000000000
119602,1,0,0,0,0,1,1,0,0,0,...,0,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,NaN,Consent provided,0
119603,1,0,1,0,0,0,1,0,0,0,...,0,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",Servicemember,NaN,259200000000000
119604,1,0,1,0,0,0,1,0,0,0,...,0,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,NaN,NaN,86400000000000


In [17]:
train.isnull().sum()

Yes                                     0
Fax                                     0
Phone                                   0
Postal mail                             0
Referral                                0
Web                                     0
Closed with explanation                 0
Closed with monetary relief             0
Closed with non-monetary relief         0
Closed with relief                      0
Closed without relief                   0
Untimely response                       0
Consumer Loan                           0
Credit card                             0
Credit reporting                        0
Debt collection                         0
Money transfers                         0
Mortgage                                0
Other financial service                 0
Payday loan                             0
Prepaid card                            0
Student loan                            0
Virtual currency                        0
Sub-product                       

In [18]:
train['Issue'].value_counts().head(12)

Loan modification,collection,foreclosure    80302
Incorrect information on credit report      58527
Loan servicing, payments, escrow account    51403
Cont'd attempts collect debt not owed       36367
Account opening, closing, or management     23568
Disclosure verification of debt             16235
Communication tactics                       15312
Deposits and withdrawals                    14721
Application, originator, mortgage broker    11201
Billing disputes                             9600
Credit reporting company's investigation     9492
Other                                        9442
Name: Issue, dtype: int64

In [19]:
train['Issue'].nunique()

95

In [20]:
top_20_i=[x for x in train['Issue'].value_counts().sort_values(ascending=False).head(12).index]

In [21]:
top_20_i

['Loan modification,collection,foreclosure',
 'Incorrect information on credit report',
 'Loan servicing, payments, escrow account',
 "Cont'd attempts collect debt not owed",
 'Account opening, closing, or management',
 'Disclosure verification of debt',
 'Communication tactics',
 'Deposits and withdrawals',
 'Application, originator, mortgage broker',
 'Billing disputes',
 "Credit reporting company's investigation",
 'Other']

In [22]:
for val in top_20_i:
    train[val]=where(train['Issue']==val,1,0)

In [23]:
xtest['Issue'].value_counts().head(12)

Loan modification,collection,foreclosure    19942
Incorrect information on credit report      14818
Loan servicing, payments, escrow account    12883
Cont'd attempts collect debt not owed        8931
Account opening, closing, or management      5934
Disclosure verification of debt              4018
Communication tactics                        3802
Deposits and withdrawals                     3762
Application, originator, mortgage broker     2813
Billing disputes                             2362
Other                                        2322
Credit reporting company's investigation     2307
Name: Issue, dtype: int64

In [24]:
xtest['Issue'].nunique()

94

In [25]:
type(xtest['Issue'].value_counts().sort_values(ascending=False).head(12).index)

pandas.core.indexes.base.Index

In [26]:
top_20_ii=[x for x in xtest['Issue'].value_counts().sort_values(ascending=False).head(12).index]

In [27]:
type(top_20_ii)

list

In [28]:
for val in top_20_ii:
    xtest[val]=where(xtest['Issue']==val,1,0)

In [29]:
xtest['Company'].nunique()

2237

In [30]:
top_10_c=[x for x in train['Company'].value_counts().sort_values(ascending=False).head(10).index]

In [31]:
for val in top_10_c:
    train[val]=where(train['Company']==val,1,0)

In [32]:
top_10_cc=[x for x in xtest['Company'].value_counts().sort_values(ascending=False).head(10).index]

In [33]:
for val in top_10_cc:
    xtest[val]=where(xtest['Company']==val,1,0)

In [34]:
print(train['Sub-product'].isnull().sum())
print(train['Sub-product'].nunique())
train['Sub-product'].value_counts().head(15)

138473
47


Other mortgage                            61522
Conventional fixed mortgage               48028
Checking account                          38310
Other (i.e. phone, health club, etc.)     25691
I do not know                             18169
Credit card                               17981
Conventional adjustable mortgage (ARM)    17566
FHA mortgage                              16316
Non-federal student loan                  15287
Medical                                   11514
Vehicle loan                              10714
Other bank product/service                 9896
Payday loan                                7837
Home equity loan or line of credit         7605
Installment loan                           4788
Name: Sub-product, dtype: int64

In [35]:
top_10_spt=[x for x in train['Sub-product'].value_counts().sort_values(ascending=False).head(10).index]

In [36]:
for val in top_10_spt:
    train[val]=where(train['Sub-product']==val,1,0)

In [38]:
top_10_sp=[x for x in xtest['Sub-product'].value_counts().sort_values(ascending=False).head(10).index]

In [39]:
for val in top_10_sp:
    xtest[val]=where(xtest['Sub-product']==val,1,0)

In [40]:
train.shape,xtest.shape

((478421, 64), (119606, 63))

In [41]:
print(train['Sub-issue'].isnull().sum())
print(train['Sub-issue'].nunique())
train['Sub-issue'].value_counts().head(15)

292625
68


Account status                              22891
Debt is not mine                            22333
Information is not mine                     17625
Not given enough info to verify debt        11124
Debt was paid                                9885
Frequent or repeated calls                   9540
Account terms                                6360
Attempted to collect wrong amount            5431
Public record                                4893
Problem getting my free annual report        4265
Personal information                         4159
Right to dispute notice not received         4085
No notice of investigation status/result     3554
Problem with statement of dispute            3552
Report improperly shared by CRC              3059
Name: Sub-issue, dtype: int64

In [42]:
top_10_sb=[x for x in train['Sub-issue'].value_counts().sort_values(ascending=False).head(10).index]

In [43]:
for val in top_10_sb:
    train[val]=where(train['Sub-issue']==val,1,0)

In [44]:
top_10_sbt=[x for x in xtest['Sub-issue'].value_counts().sort_values(ascending=False).head(10).index]

In [45]:
for val in top_10_sbt:
    xtest[val]=where(xtest['Sub-issue']==val,1,0)

In [46]:
train.shape,xtest.shape

((478421, 74), (119606, 73))

In [47]:
top_20_ccn=[x for x in train['Consumer complaint narrative'].value_counts().sort_values(ascending=False).head(20).index]

In [48]:
for val in top_20_ccn:
    train[val]=where(train['Consumer complaint narrative']==val,1,0)

In [49]:
top_10_ccn1=[x for x in xtest['Consumer complaint narrative'].value_counts().sort_values(ascending=False).head(20).index]

In [50]:
for val in top_10_ccn1:
    xtest[val]=where(xtest['Consumer complaint narrative']==val,1,0)

In [51]:
train.shape,xtest.shape

((478421, 94), (119606, 93))

In [131]:
# Consumer complaint narrative    101049
# Company public response          96830
# Company                              0
# State                              925
# ZIP code                           926
# Tags                            102735
# Consumer consent provided?       85742

In [52]:
top_10_cpr=[x for x in train['Company public response'].value_counts().sort_values(ascending=False).head(5).index]

In [53]:
for val in top_10_cpr:
    train[val]=where(train['Company public response']==val,1,0)

In [54]:
top_10_cpr1=[x for x in xtest['Company public response'].value_counts().sort_values(ascending=False).head(5).index]

In [55]:
for val in top_10_cpr1:
    xtest[val]=where(xtest['Company public response']==val,1,0)

In [56]:
train['Consumer consent provided?'].nunique()

4

In [57]:
train.shape,xtest.shape

((478421, 99), (119606, 98))

In [58]:
top_10_ccp=[x for x in train['Consumer consent provided?'].value_counts().sort_values(ascending=False).index]

In [61]:
for val in top_10_ccp:
    train[val]=where(train['Consumer consent provided?']==val,1,0)

In [62]:
top_10_ccp=[x for x in xtest['Consumer consent provided?'].value_counts().sort_values(ascending=False).index]

In [63]:
for val in top_10_ccp:
    xtest[val]=where(xtest['Consumer consent provided?']==val,1,0)

In [64]:
top_10_t=[x for x in train['Tags'].value_counts().sort_values(ascending=False).index]

In [65]:
for val in top_10_t:
    train[val]=where(train['Tags']==val,1,0)

In [66]:
top_10_t1=[x for x in xtest['Tags'].value_counts().sort_values(ascending=False).index]

In [67]:
for val in top_10_t1:
    xtest[val]=where(xtest['Tags']==val,1,0)

In [68]:
train

,Yes,Fax,Phone,Postal mail,Referral,Web,Closed with explanation,Closed with monetary relief,Closed with non-monetary relief,Closed with relief,...,Company has responded to the consumer and the CFPB and chooses not to provide a public response,Company believes it acted appropriately as authorized by contract or law,Company believes complaint caused principally by actions of third party outside the control or direction of the company,Company believes the complaint is the result of a misunderstanding,Consent provided,Consent not provided,Consent withdrawn,Older American,Servicemember,"Older American, Servicemember"
0,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478416,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
478417,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
478418,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
478419,1,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
train.columns

Index(['Yes', 'Fax', 'Phone', 'Postal mail', 'Referral', 'Web',
       'Closed with explanation', 'Closed with monetary relief',
       'Closed with non-monetary relief', 'Closed with relief',
       ...
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       'Company believes the complaint is the result of a misunderstanding',
       'Consent provided', 'Consent not provided', 'Consent withdrawn',
       'Older American', 'Servicemember', 'Older American, Servicemember'],
      dtype='object', length=105)

In [70]:
for col in ['Issue','Company','Sub-product','Consumer complaint narrative','Company public response','Consumer consent provided?','Sub-issue','Tags']:
    train=train.drop([col],axis=1)
    xtest=xtest.drop([col],axis=1)

In [71]:
ytest['Consumer disputed?'] = le.fit_transform(ytest['Consumer disputed?'])


In [72]:
ytest=ytest.drop(['Complaint ID'],axis=1)

In [73]:
train['Consumer disputed?'] = le.fit_transform(train['Consumer disputed?'])


In [74]:
xtrain = train.drop('Consumer disputed?', axis=1)
ytrain = train['Consumer disputed?']



In [75]:
xtrain.shape,xtest.shape

((478421, 96), (119606, 96))

In [76]:
from sklearn.linear_model import LogisticRegression

In [77]:
model_logistic=LogisticRegression(C = 0.0001, tol = 0.000001, class_weight = "balanced",random_state = 42)
model_logistic.fit(xtrain,ytrain)

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=1e-06, verbose=0,
                   warm_start=False)

In [78]:
cross_val_score(model_logistic,xtrain,ytrain,cv = 10 , scoring  = 'accuracy').mean()

0.5633657380681825

In [79]:
pr=model_logistic.predict(xtest)

In [80]:
auc=roc_auc_score(ytest,pr)
auc

0.7911086104930072

In [81]:
confusion_matrix(ytest,pr)

array([[68814, 49379],
       [    0,  1413]], dtype=int64)

In [82]:
accuracy_score(ytest,pr)

0.5871528184204806

In [83]:
from sklearn.metrics import confusion_matrix,classification_report
print("Classification Report")
print (classification_report(ytest,pr))
print ("Confusion matrix")
print (confusion_matrix(ytest,pr))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.58      0.74    118193
           1       0.03      1.00      0.05      1413

    accuracy                           0.59    119606
   macro avg       0.51      0.79      0.40    119606
weighted avg       0.99      0.59      0.73    119606

Confusion matrix
[[68814 49379]
 [    0  1413]]


In [84]:
#after applying pca , components are still same ...

In [82]:
#logistic is giving better results than other alg 
#checked for tree models , model preformance is low 